In [1]:
# 소프트맥스 회귀의 비용 함수 구현하기

import torch
import torch.nn.functional as F

torch.manual_seed(1)

In [3]:
# 파이토치로 소프트맥스의 비용 함수 구현하기 (로우-레벨)

# 3개의 원소를 가진 벡터 텐서를 이용해 소프트맥스 함수 이해

z = torch.FloatTensor([1, 2, 3]) # 소프트맥스 함수의 입력
hypothesis = F.softmax(z, dim=0) 
print(hypothesis)
hypothesis.sum() # 소프트맥스로 변환된 텐서의 합

tensor([0.0900, 0.2447, 0.6652])


tensor(1.)

In [4]:
# 비용 함수 구현해보기 

# 임의의 3 X 5 행렬의 크기를 가진 텐서 생성
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1) # dim=1 : 두 번째 차원에 대해 소프트맥스 함수 적용 -> 각 행의 원소의 합이 1이 되는 행렬 반환
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [5]:
# 각 샘플에 대해 임의의 레이블 생성
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [6]:
# 각 레이블에 대해 원-핫 인코딩을 수행

# 모든 원소가 0의 값을 가진 3 X 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)
# y.unsqueeze(1) 하면 (3,) 의 크기를 가진 y 텐서가 (3, 1) 크기가 된다
# scatter 의 첫 번째 인자 1 : dim = 1 ??? scatter 함수 알아보기

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [7]:
print(y.unsqueeze(1))

tensor([[0],
        [2],
        [1]])


In [8]:
print(y_one_hot)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [10]:
# 소프트맥스 회귀의 비용 함수 코드로 구현
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)


tensor(1.4689, grad_fn=<MeanBackward0>)


In [ ]:
# 파이토치로 소프트맥스의 비용 함수 구현하기 (하이-레벨)

In [11]:
# 1. F.softmax() + torch.log() = F.log_softmax()

# low-level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [12]:
# 파이토치에서는 두 개의 함수를 결합한 F.log_softmax() 도구 제공

# High-level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [14]:
# 2. F.log_softmax() + F.nll_loss() = F.cross_entropy()

# Low-Level 첫번쨰 수식
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [15]:
# Low-Level 두번쨰 수식 : log_softmax() 로 대체
(y_one_hot * -torch.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [16]:
# High-Level 세번째 수식 : nll_loss() 사용

# nll : Negative Log Likelihood
# nll_loss : F.log_softmax 를 수행한 후 남은 수식을 수행
# 원-핫 벡터를 넣을 필요없이 바로 실제값을 인자로 사용
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [17]:
# High-Level 네번째 수식 : F.cross_entropy() 사용
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)